Datasource: "Leafsnap: A Computer Vision System for Automatic Plant Species Identification,"  
Neeraj Kumar, Peter N. Belhumeur, Arijit Biswas, David W. Jacobs, W. John Kress, Ida C. Lopez, João V. B. Soares,  
Proceedings of the 12th European Conference on Computer Vision (ECCV),  
October 2012

In [1]:
tarurl = r'http://leafsnap.com/static/dataset/leafsnap-dataset.tar'

In [16]:
pwd

'C:\\Users\\User\\Documents\\GitHub\\datasets\\dataset\\images\\field'

In [2]:
%cd ../../datasets/

C:\Users\User\Documents\GitHub\datasets


In [ ]:
!curl -O $tarurl 

In [3]:
!tar -xvf leafsnap-dataset.tar

dataset/images/
dataset/images/field/
dataset/images/field/ulmus_glabra/
dataset/images/field/ulmus_glabra/13291717341164.jpg
dataset/images/field/ulmus_glabra/13291717347502.jpg
dataset/images/field/ulmus_glabra/13291717349337.jpg
dataset/images/field/ulmus_glabra/13291717347622.jpg
dataset/images/field/ulmus_glabra/13291717342603.jpg
dataset/images/field/ulmus_glabra/13291717345038.jpg
dataset/images/field/ulmus_glabra/13291717341614.jpg
dataset/images/field/ulmus_glabra/13291717345731.jpg
dataset/images/field/ulmus_glabra/13291717348788.jpg
dataset/images/field/ulmus_glabra/13291717349064.jpg
dataset/images/field/ulmus_glabra/13291717343709.jpg
dataset/images/field/ulmus_glabra/13291717344393.jpg
dataset/images/field/ulmus_glabra/13291717347291.jpg
dataset/images/field/ulmus_glabra/13291717343335.jpg
dataset/images/field/ulmus_glabra/13291717347953.jpg
dataset/images/field/ulmus_glabra/13291717348659.jpg
dataset/images/field/ulmus_glabra/13291717348826.jpg
dataset/images/field/ulmus

dataset/images/field/oxydendrum_arboreum/13001995847558.jpg
dataset/images/field/oxydendrum_arboreum/13001995843219.jpg
dataset/images/field/oxydendrum_arboreum/13001995842689.jpg
dataset/images/field/picea_orientalis/
dataset/images/field/picea_orientalis/13291692924799.jpg
dataset/images/field/picea_orientalis/13291692927394.jpg
dataset/images/field/picea_orientalis/13291692926901.jpg
dataset/images/field/picea_orientalis/13291692922679.jpg
dataset/images/field/picea_orientalis/13291692929785.jpg
dataset/images/field/picea_orientalis/13291692925493.jpg
dataset/images/field/picea_orientalis/13291692924319.jpg
dataset/images/field/picea_orientalis/13291692926347.jpg
dataset/images/field/picea_orientalis/13291692925862.jpg
dataset/images/field/picea_orientalis/13291692926293.jpg
dataset/images/field/picea_orientalis/13291692929001.jpg
dataset/images/field/picea_orientalis/13291692929494.jpg
dataset/images/field/picea_orientalis/13291692926484.jpg
dataset/images/field/picea_orientalis/13

tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


## Imports

In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import skimage
import os
import math
import random

In [18]:
%cd ./dataset/images/field/

[WinError 3] The system cannot find the path specified: './dataset/images/field/'
C:\Users\User\Documents\GitHub\datasets\dataset\images\field


In [19]:
tree_specs = os.popen('ls').read()
tree_specs = tree_specs.strip().split(sep='\n')

In [38]:
!mkdir test,train,vdate

A subdirectory or file test already exists.
Error occurred while processing: test.
A subdirectory or file train already exists.
Error occurred while processing: train.
A subdirectory or file vdate already exists.
Error occurred while processing: vdate.


In [40]:
#move images into train, test, validate folders with subfolders for classes
test_ratio = .15
for spec in tree_specs:
    %cd ./$spec
    listing = os.popen('ls').read().strip().split(sep='\n')
    random.shuffle(listing) #inplace
    im_count = len(listing)
    test_size=val_size = math.ceil(test_ratio*im_count)
    train_size = im_count - (test_size + val_size)
    subfolders = ['train', 'test', 'vdate']
    for subfolder in subfolders:
        os.makedirs(os.path.join(os.path.dirname(os.getcwd()), subfolder, spec))
    for item in range(0, train_size):
        moved = listing.pop()
        !mv $moved ../train/$spec/$moved
    for item in range(0, test_size):
        moved = listing.pop()
        !mv $moved ../test/$spec/$moved
    for item in range(0, val_size):
        moved = listing.pop()
        !mv $moved ../vdate/$spec/$moved
    %cd ..
    !rmdir ./$spec

[WinError 2] The system cannot find the file specified: './acer_platanoides'
C:\Users\User\Documents\GitHub\datasets\dataset\images\field\acer_platanoides
train file to move is 1249060544_0001.jpg
train file to move is 1259110850_0062.jpg
train file to move is 1259110850_0066.jpg
train file to move is 1249316899_0003.jpg
train file to move is 1259110850_0051.jpg
train file to move is 1249316899_0000.jpg
train file to move is 1258491080_0003.jpg
train file to move is 1249316899_0004.jpg
train file to move is 1258491080_0000.jpg
train file to move is 1258491080_0004.jpg
train file to move is 1249060544_0000.jpg
train file to move is 1258491080_0002.jpg
train file to move is 1249060544_0002.jpg
C:\Users\User\Documents\GitHub\datasets\dataset\images\field
C:\Users\User\Documents\GitHub\datasets\dataset\images\field\acer_saccharinum
train file to move is 13001158006219.jpg
train file to move is 13001158004356.jpg
train file to move is 13001158000103.jpg
train file to move is 13001158000494.

train file to move is 13001271783287.jpg
train file to move is 1249059957_0004.jpg
train file to move is 13001271785047.jpg
train file to move is 13001271786337.jpg
train file to move is 13001271780941.jpg
train file to move is 13001271785409.jpg
train file to move is 13001271787084.jpg
train file to move is 13001271783188.jpg
train file to move is 1259110850_0115.jpg
train file to move is 13001271780474.jpg
train file to move is 1249316343_0002.jpg
train file to move is 1249316343_0001.jpg
train file to move is 13001271787763.jpg
train file to move is 13001271781307.jpg
train file to move is 1249316343_0003.jpg
train file to move is 13001271786674.jpg
train file to move is 13001271785510.jpg
train file to move is 13001271784201.jpg
train file to move is 13001271784976.jpg
C:\Users\User\Documents\GitHub\datasets\dataset\images\field
C:\Users\User\Documents\GitHub\datasets\dataset\images\field\chamaecyparis_thyoides
train file to move is 13001300968521.jpg
train file to move is 13001300

train file to move is 13291804052573.jpg
train file to move is 13291804050836.jpg
train file to move is 13291804059300.jpg
train file to move is 13291804053577.jpg
train file to move is 13291804058755.jpg
train file to move is 13291804057391.jpg
train file to move is 13291804053186.jpg
train file to move is 13291804051085.jpg
train file to move is 13291804056291.jpg
train file to move is 13291804056913.jpg
train file to move is 13291804056458.jpg
train file to move is 13291804050219.jpg
train file to move is 13291804050846.jpg
train file to move is 13291804051263.jpg
train file to move is 13291804057356.jpg
train file to move is 13291804059775.jpg
train file to move is 13291804052199.jpg
train file to move is 13291804055083.jpg
train file to move is 13291804053025.jpg
train file to move is 13291804051608.jpg
train file to move is 13291804059710.jpg
train file to move is 13291804052925.jpg
train file to move is 13291804055690.jpg
train file to move is 13291804053497.jpg
train file to mo

train file to move is 13001959994158.jpg
train file to move is 13001959996778.jpg
train file to move is 13001959993586.jpg
train file to move is 13001959995338.jpg
train file to move is 13001959990829.jpg
train file to move is 13001959996927.jpg
train file to move is 13001959992634.jpg
train file to move is 13001959991177.jpg
train file to move is 13001959991873.jpg
train file to move is 13001959998816.jpg
train file to move is 13001959994838.jpg
train file to move is 13001959994192.jpg
train file to move is 13001959994984.jpg
train file to move is 13001959995519.jpg
train file to move is 13001959998712.jpg
C:\Users\User\Documents\GitHub\datasets\dataset\images\field
C:\Users\User\Documents\GitHub\datasets\dataset\images\field\magnolia_soulangiana
train file to move is 1252647480_0002.jpg
train file to move is 1252647480_0003.jpg
train file to move is 1252647480_0001.jpg
train file to move is 1249580270_0002.jpg
train file to move is 1259118878_0010.jpg
train file to move is 1259118878

train file to move is 13291692923333.jpg
train file to move is 13291692923710.jpg
train file to move is 13291692927714.jpg
train file to move is 13291692929494.jpg
train file to move is 13291692926605.jpg
train file to move is 13291692929001.jpg
train file to move is 13291692925986.jpg
train file to move is 13291692925493.jpg
train file to move is 13291692922763.jpg
train file to move is 13291692927394.jpg
train file to move is 13291692920150.jpg
train file to move is 13291692927906.jpg
train file to move is 13291692926949.jpg
train file to move is 13291692926901.jpg
train file to move is 13291692920291.jpg
train file to move is 13291692926293.jpg
train file to move is 13291692921010.jpg
train file to move is 13291692926172.jpg
train file to move is 13291692926539.jpg
train file to move is 13291692929600.jpg
train file to move is 13291692927458.jpg
train file to move is 13291692922593.jpg
train file to move is 13291692924666.jpg
train file to move is 13291692929650.jpg
train file to mo

C:\Users\User\Documents\GitHub\datasets\dataset\images\field
C:\Users\User\Documents\GitHub\datasets\dataset\images\field\quercus_falcata
train file to move is 12992006383179.jpg
train file to move is 12992006348202.jpg
train file to move is 12992006528317.jpg
train file to move is 12992006508897.jpg
train file to move is 12992006513880.jpg
train file to move is 12992006266923.jpg
train file to move is 12992006143472.jpg
train file to move is 12992006063047.jpg
train file to move is 12992006456131.jpg
train file to move is 12992006243184.jpg
train file to move is 12992006463661.jpg
train file to move is 12992006277263.jpg
train file to move is 12992006067496.jpg
train file to move is 12992006130485.jpg
train file to move is 12992006211205.jpg
train file to move is 12992006202697.jpg
train file to move is 12992006399764.jpg
train file to move is 12992006195683.jpg
train file to move is 12992006400956.jpg
train file to move is 12992006353516.jpg
train file to move is 12992006438621.jpg
t

train file to move is 13292245043719.jpg
train file to move is 13292254750759.jpg
train file to move is 13292254758770.jpg
train file to move is 13292245044126.jpg
train file to move is 13292245042577.jpg
train file to move is 13292254753836.jpg
train file to move is 13292254757543.jpg
train file to move is 13292245041958.jpg
train file to move is 13292254750709.jpg
train file to move is 13292245045228.jpg
train file to move is 13292245048665.jpg
train file to move is 13292245045895.jpg
train file to move is 13292254759214.jpg
train file to move is 13292245042320.jpg
train file to move is 13292254752655.jpg
train file to move is 13292254753760.jpg
train file to move is 13292245045249.jpg
train file to move is 13292254751847.jpg
train file to move is 13292245048753.jpg
train file to move is 13292245041899.jpg
train file to move is 13292254759233.jpg
train file to move is 13292245042767.jpg
train file to move is 13292245047861.jpg
train file to move is 13292245043692.jpg
train file to mo

train file to move is 13002212277663.jpg
train file to move is 13002212279993.jpg
train file to move is 13002212277828.jpg
train file to move is 13002212276945.jpg
train file to move is 13002212275801.jpg
train file to move is 13002212270403.jpg
train file to move is 13002212271936.jpg
train file to move is 13002212274066.jpg
train file to move is 13002212279536.jpg
train file to move is 13002212278764.jpg
train file to move is 13002212276540.jpg
train file to move is 13002212278698.jpg
train file to move is 13002212276069.jpg
train file to move is 13002212273899.jpg
train file to move is 13002212275099.jpg
train file to move is 13002212273714.jpg
train file to move is 13002212278130.jpg
train file to move is 13002212273924.jpg
train file to move is 13002212273591.jpg
train file to move is 13002212274763.jpg
train file to move is 13002212274913.jpg
train file to move is 13002212277423.jpg
train file to move is 13002212273174.jpg
train file to move is 13002212272028.jpg
C:\Users\User\Do

In [41]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
vdate_datagen = ImageDataGenerator(rescale=1/255)

In [61]:
# Flow training images in batches of 128 using train_datagen generator
batch_size = 32
target_size = (256,256)
train_generator = train_datagen.flow_from_directory(
        './train/',  # This is the source directory for training images
        target_size=target_size, 
        batch_size=batch_size,
        # Specify the classes explicitly
        classes = tree_specs,
        class_mode='categorical')

Found 1303 images belonging to 48 classes.


In [62]:
test_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=target_size,
        batch_size=batch_size,
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


In [63]:
vdate_generator = vdate_datagen.flow_from_directory(
        './vdate/',
        target_size=target_size,
        batch_size=batch_size,
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


# Modeling
### Sequential Model

In [96]:
sequential_mod = tf.keras.models.Sequential()
sequential_mod.add(Dense(64, activation='relu', input_shape=(256,256,3)))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(64, activation='relu'))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(48, activation='softmax'))

sequential_mod.summary()

sequential_mod.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 256, 256, 64)      256       
_________________________________________________________________
dropout_32 (Dropout)         (None, 256, 256, 64)      0         
_________________________________________________________________
dense_56 (Dense)             (None, 256, 256, 64)      4160      
_________________________________________________________________
dropout_33 (Dropout)         (None, 256, 256, 64)      0         
_________________________________________________________________
dense_57 (Dense)             (None, 256, 256, 48)      3120      
Total params: 7,536
Trainable params: 7,536
Non-trainable params: 0
_________________________________________________________________


In [97]:
total_sample = train_generator.n

In [99]:
n_epochs = 10
history = sequential_mod.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

Epoch 1/10


ValueError: A target array with shape (32, 48) was passed for an output of shape (None, 256, 256, 48) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.